We first need to find the images by outputting a list of galaxy names, which I just outputted from the main list's galaxy name column. Then we need to input the list of name into a simple .txt file so that the website can read in the list.

http://archives.esac.esa.int/hsa/whsa/ - Using this website, we can input the .txt file into the targets file section.

We still have to manually select the .fits file we want to download, which we can do by only checking for the PACS Photometry in the Instrument tab of the Observation Contraint. As well as search for only Extra-Galactic objects in the Object type.

Once we have the collection of .fits images, we now need to manually sort them by galaxy names, as well as the wavelength the image is at.

I've included the sorted .fits file within the galaxy folder of the zip file. Along with other files mentioned later on.

Once we have the base .fits files, we can start extracting data from them and perform photometry, where I've done in the photometry.ipynb notebook.

The notebook mainly does 3 things : 

First perform photometry by finding the center of the galaxy using the RA and DEC, then convert it into pixels via WCS of the .fits image. It then creates its own ellipses, and then make progressively bigger ellipses until it thinks it's captured most of the light within the galaxy. Furthermore, it also calculates the surface brightness of the image by finding the difference of two consecutive ellipses. This is outputted as a poster of plots.

Secondly, the notebook takes in the AP06 parameters (semi-major axis, epillicity, rotational angle, etc) of these galaxies from the ephot .fits file, and then create an ellipse that is overlaid on the image to check for anormalies (which is exported as .png files). Then it calculates the flux of the galaxy within that ellipse. Then, it outputs the flux result into a table by their wavelengths

And finally, it creates individual .html webpages for the galaxies, as well as a few plots of the resultant fluxes.

Once we have the individual html websites, we can then create a central hub using this code :

In [ ]:
#Import the necessary packages

from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
from astropy.io.ascii import masked
from astropy.io import ascii
import glob
from astropy.io import fits
import wget
import matplotlib.image as mpimg
from astropy.wcs import WCS
from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm
from reproject import reproject_interp
import sys
from IPython.display import clear_output
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization import ImageNormalize
from astropy.visualization import LogStretch
from astropy.wcs import WCS
import glob
import os
from scipy.stats import scoreatpercentile
import astropy.units as u
from IPython.display import clear_output
from matplotlib import colors
import warnings
warnings.filterwarnings('ignore')

mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

#Set the home path

os.environ['HOME'] ='C:/Users/USER/Documents/GitHub'
homedir = os.getenv("HOME")
tabledir = homedir+'/Virgo/tables'
plotdir = homedir+'/Virgo/plots'
datadir = homedir+'/HTML-building/galaxy'
def imdisplay(image, v1perc=10, v2perc=95, logscale=True):
    '''
    display an image 
    OPTIONAL KEYWORD PARAMETERS
    v1perc: one end of the colormap assigned to the v1perc percent lowest flux 
    v2perc: the other end of the colormap assigned to the v2perc percent highest flux    
    '''
    # make sure image is an np array
    nimage = np.array(image)
    # determine the pixel values at the 10th and 95th percentile
    v1 = scoreatpercentile(nimage,v1perc)
    v2 = scoreatpercentile(nimage,v2perc)
    # display using imshow
    #
    # you can play with alternate cmaps in the function below, such as "viridis" or "gray"
    # The 'gray_r' color map reverses the color-scale so that dark display pixels are the brightest in the image
    #
    # vmin and vmax set the min and max pixel values that
    # will be mapped to the extremes of the colormap
    print(v1,v2)
    norm = None
    if (logscale):
        norm = ImageNormalize(vmin=v1, vmax=v2, stretch=LogStretch())
    else:
        norm = ImageNormalize(vmin=v1, vmax=v2)
    im = ax.imshow(image, origin='lower', norm=norm)
    plt.axis('off')    

    return fig,ax
    #fig.colorbar(fraction=.08)
def find_files(destination_folder, partial_name):
    matching_files = []

    for root, dirs, files in os.walk(destination_folder):
        for file in files:
            if partial_name.lower() in file.lower():
                matching_files.append(os.path.join(root, file))

    return matching_files 

In [ ]:
#Generates images from the .fits files
csv_file = homedir+'/HTML-building/Herschelstuff.csv'
galaxy = Table.read(csv_file)
for i in range(len(galaxy)):
    galaxy_name = str(galaxy['name'][i])
    path =  datadir+'/pipeline/'+galaxy_name
    VFID = str(galaxy['vfid_01'][i])
    if os.path.exists(path):
        destination_folder = path+'\\HPPUNIMAPB'
        partial_name = 'hpacs_25HPPUNIMAPB'
        found_files = find_files(destination_folder, partial_name)
        if found_files:
            found_file = found_files[0]
            fig,ax = plt.subplots(figsize=(10,10))
            data, header = fits.getdata(found_file, header=True)
            nimage = np.array(data)
            v1 = None
            v2 = None
            for v1perc in np.arange(0.1, 40.0, 0.1):
                v1 = scoreatpercentile(nimage, v1perc)
                if v1 > 0:
                    break
            for v2perc in np.arange(99.9, 20.0, -0.1):
                v2 = scoreatpercentile(nimage, v2perc)
                if not np.isnan(v2):
                    break
            imdisplay(data,v1perc,v2perc,logscale=True)
            plt.subplots_adjust(wspace=0, hspace=0)
            plt.savefig(datadir+'/png/'+VFID+'-'+galaxy_name+'blue.png', dpi=150)
            plt.close()
        destination_folder = path+'\\HPPUNIMAPG'
        partial_name = 'hpacs_25HPPUNIMAPB'
        found_files = find_files(destination_folder, partial_name)
        if found_files:
            found_file = found_files[0]
            fig,ax = plt.subplots(figsize=(10,10))
            data, header = fits.getdata(found_file, header=True)
            nimage = np.array(data)
            v1 = None
            v2 = None
            for v1perc in np.arange(0.1, 40.0, 0.1):
                v1 = scoreatpercentile(nimage, v1perc)
                if v1 > 0:
                    break
            for v2perc in np.arange(99.9, 20.0, -0.1):
                v2 = scoreatpercentile(nimage, v2perc)
                if not np.isnan(v2):
                    break
            imdisplay(data,v1perc,v2perc,logscale=True)
            plt.subplots_adjust(wspace=0, hspace=0)
            plt.savefig(datadir+'/png/'+VFID+'-'+galaxy_name+'green.png', dpi=150)
            plt.close()
        destination_folder = path+'\\HPPUNIMAPR'
        partial_name = 'hpacs_25HPPUNIMAPR'
        found_files = find_files(destination_folder, partial_name)
        if found_files:
            found_file = found_files[0]
            fig,ax = plt.subplots(figsize=(10,10))
            data, header = fits.getdata(found_file, header=True)
            nimage = np.array(data)
            v1 = None
            v2 = None
            for v1perc in np.arange(0.1, 40.0, 0.1):
                v1 = scoreatpercentile(nimage, v1perc)
                if v1 > 0:
                    break
            for v2perc in np.arange(99.9, 20.0, -0.1):
                v2 = scoreatpercentile(nimage, v2perc)
                if not np.isnan(v2):
                    break
            imdisplay(data,v1perc,v2perc,logscale=True)
            plt.subplots_adjust(wspace=0, hspace=0)
            plt.savefig(datadir+'/png/'+VFID+'-'+galaxy_name+'red.png', dpi=150)
            plt.close()
    else:
        print(f"Galaxy not found: {VFID}")
        print(galaxy_name)
print('done')

In [ ]:
#Generate the home website
csv_file = homedir+'/HTML-building/Herschelstuff.csv'
galaxy = Table.read(csv_file)
pixscale=1   #standard
pscale = 0
xsize = 0
for i in range(len(galaxy)):
    galaxy_name = str(galaxy['name'][i])
    path =  datadir+'/pipeline/'+galaxy_name
    destination_folder = path+'/HPPUNIMAPR/'
    partial_name = 'hpacs_25HPPUNIMAPR'
    found_files = find_files(destination_folder, partial_name)
    VFID = str(galaxy['vfid_01'][i])
    filename_LS = datadir+'/png/'+VFID+'-'+galaxy_name+'-LS.jpg'
    RA = galaxy['ra_01'][i]
    DEC = galaxy['dec_01'][i]
    if os.path.exists(path):
        if found_files:
            found_file = found_files[0]
            image, head = fits.getdata(found_file, header=True)
            wcs_info = WCS(head) 
            pscale=np.abs(float(head['CDELT1']))   #grab transformation matrix of Herschel image
            xsize=np.abs(int(head['NAXIS1']))   #grab length of Herschel image
            xsize_arcsec=pscale*3600*xsize   #length convert to arcseconds
            imsize=int(xsize_arcsec/pixscale)   #convert length to an integer
            imsize=str(imsize)   #convert integer length to a...string       
            image_url = f'https://www.legacysurvey.org/viewer/cutout.jpg?ra={RA}&dec={DEC}&layer=ls-dr9&size={imsize}&pixscale={1}'
            if os.path.exists(filename_LS):
                os.remove(filename_LS)
                imageLS = wget.download(image_url,out=filename_LS)
            else:
                imageLS = wget.download(image_url,out=filename_LS)
    else:
        print(f"Galaxy not found: {VFID}-{galaxy_name}")
print('done')

Furthermore, the link to the files should be at https://drive.google.com/file/d/1OToTzadUD08iQCq3gcZEACynV1jv9uQd/view?usp=sharing